In [1]:
from synthcity.metrics.eval import Metrics
import pandas as pd
def synthcity_evaluate(real_path,
                       syn_path,
                       result_name=None,
                       real_size = None,
                       syn_size = None): 
    real_data = pd.read_csv(real_path)
    syn_data = pd.read_csv(syn_path)
    
    if real_size and real_size < len(real_data):
        real_data = real_data.sample(n=real_size, random_state=42)  # Adjust random_state as desired for reproducibility
    if syn_size and syn_size < len(syn_data):
        syn_data = syn_data.sample(n=syn_size, random_state=42)
    
    synthcity_result = Metrics.evaluate(real_data, syn_data)
    if result_name:
        synthcity_result.to_csv(result_name)
    return synthcity_result
    
def compare_results(
        dataset1_result_path=None, 
        dataset2_result_path=None,
        dataset1_result=None,
        dataset2_result=None
        ):
    if dataset1_result_path:
        dataset1_result = pd.read_csv(dataset1_result_path)[['Unnamed: 0', 'min', 'direction']]
    if dataset2_result_path:
        dataset2_result = pd.read_csv(dataset2_result_path)[['Unnamed: 0', 'min', 'direction']]
    
    # Merge dataset1_result and dataset2_result based on the 'Unnamed: 0' column
    merged_result = pd.merge(dataset1_result, dataset2_result[['Unnamed: 0', 'min']], on='Unnamed: 0', suffixes=('_dataset1', '_dataset2'))
    
    # Reorder columns in merged_result
    # Get a list of columns excluding 'direction'
    columns_reordered = [col for col in merged_result.columns if col != 'direction']
    
    # Append 'direction' column to the end of the list
    columns_reordered.append('direction')
    
    # Reassign the DataFrame with reordered columns
    merged_result = merged_result[columns_reordered]
    return merged_result

[KeOps] Warning : omp.h header is not in the path, disabling OpenMP. To fix this, you can set the environment
                  variable OMP_PATH to the location of the header before importing keopscore or pykeops,
                  e.g. using os.environ: import os; os.environ['OMP_PATH'] = '/path/to/omp/header'
[KeOps] Warning : Cuda libraries were not detected on the system or could not be loaded ; using cpu only mode


In [7]:
samples_dir = "/Users/minhkau/Documents/TUDelft/Year 3/RP/Code/tabular-gpt/samples"
real_path = samples_dir+"/adult.csv"
synth_no_watermark_path = samples_dir+"/adult_samples_10k.csv"
synth_with_watermark_path = samples_dir+"/great_watermaked_adult_samples_1000_fixed_seed.csv"
df1 = synthcity_evaluate(real_path, real_path, real_size=1000, syn_size=1000, result_name="real_synthcity_result_1000.csv")
df2 = synthcity_evaluate(real_path, synth_no_watermark_path, real_size=1000, syn_size=1000, result_name="no_watermark_synthcity_result_1000.csv")
df3 = synthcity_evaluate(real_path, synth_with_watermark_path, real_size=1000, syn_size=1000, result_name="with_watermark_synthcity_result_1000.csv")



In [15]:
synthicity_eval_result_dir = "/Users/minhkau/Documents/TUDelft/Year 3/RP/Code/tabular-gpt/evaluation"
real_result_path = synthicity_eval_result_dir + "/real_synthcity_result.csv"
syn_result_path = synthicity_eval_result_dir + "/great_watermarked_synthicity_result.csv"
compare_df = compare_results(real_result_path, syn_result_path)

,Unnamed: 0,min_dataset1,min_dataset2,direction
0,sanity.data_mismatch.score,0.000000,1.250000e-01,minimize
1,sanity.common_rows_proportion.score,0.998700,0.000000e+00,minimize
2,sanity.nearest_syn_neighbor_distance.mean,0.000000,2.870390e-03,minimize
3,sanity.close_values_probability.score,1.000000,9.990000e-01,maximize
4,sanity.distant_values_probability.score,0.000000,1.000000e-03,minimize
5,stats.jensenshannon_dist.marginal,0.000000,2.139866e-02,minimize
6,stats.chi_squared_test.marginal,1.000000,7.956620e-01,maximize
7,stats.inv_kl_divergence.marginal,1.000000,8.381935e-01,maximize
8,stats.ks_test.marginal,1.000000,8.465333e-01,maximize
9,stats.max_mean_discrepancy.joint,0.000000,2.000011e-03,minimize


In [68]:
# df1_filter = df1[[0, 'min', 'direction']]

df1.iloc[:, 0]
df2["alo"] = df1["min"]
df2

# df1_filte
# final_df.rename(columns={'min': 'Real Data'}, inplace=True)
# final_df.columns=["Metrics", "as"]
no_water_mark_df = pd.read_csv("/Users/minhkau/Documents/TUDelft/Year 3/RP/Code/tabular-gpt/evaluation/no_watermark_synthcity_result_1000.csv")[["metrics","min"]]
with_watermark_df = pd.read_csv("/Users/minhkau/Documents/TUDelft/Year 3/RP/Code/tabular-gpt/evaluation/with_watermark_synthcity_result_1000.csv")[["metrics","min","direction"]]
real_df = pd.read_csv("/Users/minhkau/Documents/TUDelft/Year 3/RP/Code/tabular-gpt/evaluation/real_synthcity_result_1000.csv")[["metrics","min"]]

    # merged_result = pd.merge(dataset1_result, dataset2_result[['Unnamed: 0', 'min']], on='Unnamed: 0', suffixes=('_dataset1', '_dataset2'))


merge_result = pd.merge(real_df, no_water_mark_df, on="metrics", suffixes=("1","2"))
merge_result_final = pd.merge(merge_result, with_watermark_df, on="metrics", suffixes=("2","3"))
merge_result_final.columns = ["Metrics", "Real Data", "No-Watermark", "With-Watermark", "Direction"]
merge_result_final = merge_result_final.round(2)

interested_metrics = [
 "sanity.nearest_syn_neighbor_distance.mean", 
    "stats.ks_test.marginal",
    "performance.linear_model.gt",
    "performance.xgb.gt",
    "performance.mlp.gt"
]
rename_metrics = [
    "Nearest Neighbors Distance",
    "KS test"
    "Linear Model Performance",
    "XGB Performance",
    "MLP Performance"
]

# Create a mapping dictionary from old names to new names
# rename_map = dict(zip(interested_metrics, rename_metrics))


# Display the updated DataFrame
merge_result_final = merge_result_final.loc[merge_result_final["Metrics"].isin(interested_metrics)]
merge_result_final.to_csv("ditme.csv")
# for i in range(0, len(interested_metrics) - 1):
#     merge_result_final.loc[merge_result_final["Metrics"] == interested_metrics[i]] = rename_metrics[i]

merge_result_final

,Metrics,Real Data,No-Watermark,With-Watermark,Direction
2,sanity.nearest_syn_neighbor_distance.mean,0.00,0.02,0.01,minimize
8,stats.ks_test.marginal,1.00,0.78,0.72,maximize
21,performance.linear_model.gt,0.79,0.79,0.79,maximize
24,performance.mlp.gt,0.50,0.50,0.50,maximize
27,performance.xgb.gt,0.87,0.87,0.87,maximize
